In [1]:
%matplotlib inline

import torch
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary

In [55]:
#Select device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 10
batchsize = 16
batchsize_test = 16
learning_rate = 0.001
momentum = 0.5
log_interval = 20
TRAIN_DATA_PATH = "../Data/CATS_DOGS/train"
TEST_DATA_PATH = "../Data/CATS_DOGS/test"

TRANSFORM_IMG = transforms.Compose([
    transforms.Resize(150),
    transforms.CenterCrop(150),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225] )
    ])

train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=TRANSFORM_IMG)
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batchsize, shuffle=True,  num_workers=4)

test_data = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform=TRANSFORM_IMG)
test_data_loader  = torch.utils.data.DataLoader(test_data, batch_size=batchsize_test, shuffle=True, num_workers=4) 



In [56]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 20, kernel_size=3)
        self.conv2 = torch.nn.Conv2d(20, 32, kernel_size=3)
        self.conv3 = torch.nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = torch.nn.Linear((64*17*17), 50)
        self.fc2 = torch.nn.Linear(50, 1)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = x.view(-1, (64*17*17))
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.sigmoid(x)

In [57]:
#Network model
network = Network()
network.to(device)

#Stochastic gradient decent
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

criterion = torch.nn.BCELoss()

In [58]:
summary(network, (3, 150, 150))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 20, 148, 148]             560
            Conv2d-2           [-1, 32, 72, 72]           5,792
            Conv2d-3           [-1, 64, 34, 34]          18,496
            Linear-4                   [-1, 50]         924,850
            Linear-5                    [-1, 1]              51
Total params: 949,749
Trainable params: 949,749
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.26
Forward/backward pass size (MB): 5.17
Params size (MB): 3.62
Estimated Total Size (MB): 9.05
----------------------------------------------------------------


In [6]:
##Just for testign
test = iter(train_data_loader)
img, target= next(test)

#out = network(img)
#test_loss += F.binary_cross_entropy(output, target).item()
#plt.imshow(img[9][0])



In [59]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_data_loader.dataset) for i in range(num_epochs + 1)]

In [60]:
def test():
    #Sets the module in evaluation mode
    network.eval()
    test_loss = 0
    correct = 0
    #correct.to(device)
    
    #dont update dynamic computation graph
    with torch.no_grad():
        #for every example in test
        for data, target in test_data_loader:
            
            target = target.view(-1, 1).float()
            target.to(device)
            
            #evaluate the model
            output = network(data.to(device))
            
            #acumulate the loss
            test_loss += F.binary_cross_entropy(output, target.to(device)).item()
            
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred).to(device)).sum()
        
    test_loss /= len(test_data_loader.dataset)
    test_losses.append(test_loss)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_data_loader.dataset), 100. * correct / len(test_data_loader.dataset)))

In [40]:
##Just for testign
test_loss = 0
correct = 0
test = iter(test_data_loader)
img, target= next(test)

device

In [62]:
#train

test()

for epoch in range(1, num_epochs+1):
    
    for batch_idx, (data, target) in enumerate(train_data_loader):
        
        network.train()
        
        #print(f'b {batch}')
        
        #forward pass
        out = network(data.to(device))
        
        #Use negative log likelihood loss.
        loss = criterion(out, target.view(-1, 1).float().to(device))
        
        
        #with this gradients are calculated
        loss.backward()
    
        #update gradients
        optimizer.step()
        
        #Set gradients to zero
        optimizer.zero_grad()

        #Display iteration statistics
        if batch_idx % log_interval == 0:
            
            #print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_data_loader.dataset),100. * batch_idx / len(train_data_loader), loss.item()))
        
    test()
    
    


Test set: Avg. loss: 0.0415, Accuracy: 3126/6251 (50%)



/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:822: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 1 [0/18743 (0%)]	Loss: 0.666133
Train Epoch: 1 [320/18743 (2%)]	Loss: 0.749524
Train Epoch: 1 [640/18743 (3%)]	Loss: 0.665712
Train Epoch: 1 [960/18743 (5%)]	Loss: 0.686443
Train Epoch: 1 [1280/18743 (7%)]	Loss: 0.627932
Train Epoch: 1 [1600/18743 (9%)]	Loss: 0.656071
Train Epoch: 1 [1920/18743 (10%)]	Loss: 0.755791
Train Epoch: 1 [2240/18743 (12%)]	Loss: 0.603863
Train Epoch: 1 [2560/18743 (14%)]	Loss: 0.608701
Train Epoch: 1 [2880/18743 (15%)]	Loss: 0.669664
Train Epoch: 1 [3200/18743 (17%)]	Loss: 0.589374
Train Epoch: 1 [3520/18743 (19%)]	Loss: 0.625937
Train Epoch: 1 [3840/18743 (20%)]	Loss: 0.622465
Train Epoch: 1 [4160/18743 (22%)]	Loss: 0.692129
Train Epoch: 1 [4480/18743 (24%)]	Loss: 0.612354
Train Epoch: 1 [4800/18743 (26%)]	Loss: 0.739332
Train Epoch: 1 [5120/18743 (27%)]	Loss: 0.720459
Train Epoch: 1 [5440/18743 (29%)]	Loss: 0.635542
Train Epoch: 1 [5760/18743 (31%)]	Loss: 0.623815
Train Epoch: 1 [6080/18743 (32%)]	Loss: 0.670303
Train Epoch: 1 [6400/18743 (34%)

/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:822: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 2 [15360/18743 (82%)]	Loss: 0.639707
Train Epoch: 2 [15680/18743 (84%)]	Loss: 0.702309
Train Epoch: 2 [16000/18743 (85%)]	Loss: 0.560444
Train Epoch: 2 [16320/18743 (87%)]	Loss: 0.533139
Train Epoch: 2 [16640/18743 (89%)]	Loss: 0.494855
Train Epoch: 2 [16960/18743 (90%)]	Loss: 0.608043
Train Epoch: 2 [17280/18743 (92%)]	Loss: 0.616230
Train Epoch: 2 [17600/18743 (94%)]	Loss: 0.623104
Train Epoch: 2 [17920/18743 (96%)]	Loss: 0.597788
Train Epoch: 2 [18240/18743 (97%)]	Loss: 0.683515
Train Epoch: 2 [18560/18743 (99%)]	Loss: 0.687255

Test set: Avg. loss: 0.0383, Accuracy: 3126/6251 (50%)

Train Epoch: 3 [0/18743 (0%)]	Loss: 0.639854
Train Epoch: 3 [320/18743 (2%)]	Loss: 0.585417
Train Epoch: 3 [640/18743 (3%)]	Loss: 0.636801
Train Epoch: 3 [960/18743 (5%)]	Loss: 0.640155
Train Epoch: 3 [1280/18743 (7%)]	Loss: 0.530076
Train Epoch: 3 [1600/18743 (9%)]	Loss: 0.561936
Train Epoch: 3 [1920/18743 (10%)]	Loss: 0.668092
Train Epoch: 3 [2240/18743 (12%)]	Loss: 0.651317
Train Epoch: 

/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:822: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 3 [8640/18743 (46%)]	Loss: 0.619508
Train Epoch: 3 [8960/18743 (48%)]	Loss: 0.590045
Train Epoch: 3 [9280/18743 (49%)]	Loss: 0.550440
Train Epoch: 3 [9600/18743 (51%)]	Loss: 0.573702
Train Epoch: 3 [9920/18743 (53%)]	Loss: 0.670582
Train Epoch: 3 [10240/18743 (55%)]	Loss: 0.560448
Train Epoch: 3 [10560/18743 (56%)]	Loss: 0.479950
Train Epoch: 3 [10880/18743 (58%)]	Loss: 0.826532
Train Epoch: 3 [11200/18743 (60%)]	Loss: 0.640581
Train Epoch: 3 [11520/18743 (61%)]	Loss: 0.605775
Train Epoch: 3 [11840/18743 (63%)]	Loss: 0.577162
Train Epoch: 3 [12160/18743 (65%)]	Loss: 0.698808
Train Epoch: 3 [12480/18743 (67%)]	Loss: 0.555147
Train Epoch: 3 [12800/18743 (68%)]	Loss: 0.491464
Train Epoch: 3 [13120/18743 (70%)]	Loss: 0.573091
Train Epoch: 3 [13440/18743 (72%)]	Loss: 0.460550
Train Epoch: 3 [13760/18743 (73%)]	Loss: 0.635044
Train Epoch: 3 [14080/18743 (75%)]	Loss: 0.704661
Train Epoch: 3 [14400/18743 (77%)]	Loss: 0.517779
Train Epoch: 3 [14720/18743 (78%)]	Loss: 0.493388
Train

/opt/conda/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:822: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Epoch: 4 [13120/18743 (70%)]	Loss: 0.617250
Train Epoch: 4 [13440/18743 (72%)]	Loss: 0.570839
Train Epoch: 4 [13760/18743 (73%)]	Loss: 0.696942
Train Epoch: 4 [14080/18743 (75%)]	Loss: 0.592898
Train Epoch: 4 [14400/18743 (77%)]	Loss: 0.692704
Train Epoch: 4 [14720/18743 (78%)]	Loss: 0.636555
Train Epoch: 4 [15040/18743 (80%)]	Loss: 0.555562
Train Epoch: 4 [15360/18743 (82%)]	Loss: 0.482057
Train Epoch: 4 [15680/18743 (84%)]	Loss: 0.737146
Train Epoch: 4 [16000/18743 (85%)]	Loss: 0.531677
Train Epoch: 4 [16320/18743 (87%)]	Loss: 0.465667
Train Epoch: 4 [16640/18743 (89%)]	Loss: 0.555666
Train Epoch: 4 [16960/18743 (90%)]	Loss: 0.651579
Train Epoch: 4 [17280/18743 (92%)]	Loss: 0.689123
Train Epoch: 4 [17600/18743 (94%)]	Loss: 0.784079
Train Epoch: 4 [17920/18743 (96%)]	Loss: 0.636410
Train Epoch: 4 [18240/18743 (97%)]	Loss: 0.608438
Train Epoch: 4 [18560/18743 (99%)]	Loss: 0.531386


KeyboardInterrupt: 

In [28]:
print("torch.cuda.memory_allocated: %fMB"%(torch.cuda.memory_allocated(0)/1024/1024))
print("torch.cuda.memory_reserved: %fMB"%(torch.cuda.memory_reserved(0)/1024/1024))
print("torch.cuda.max_memory_reserved: %fMB"%(torch.cuda.max_memory_reserved(0)/1024/1024))


torch.cuda.memory_allocated: 166.925293MB
torch.cuda.memory_reserved: 230.000000MB
torch.cuda.max_memory_reserved: 230.000000MB


### ToDo

- Check why test validation is not working
- Check how accuracy is calculated
- Create a section in which the network is described